In [4]:
import iris
import time
import os

namespace="USER"
port = 1972
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
connection_string = f"{hostname}:{port}/{namespace}"
username = "demo"
password = "demo"

In [5]:
# Note: Ideally conn and cursor should be used with context manager or with try-execpt-finally 
conn = iris.connect(connection_string, username, password)
cursor = conn.cursor()

In [6]:
users_table_name = "Users"
users_table_definition = """(
    user_id INTEGER PRIMARY KEY,
    name VARCHAR(255),
    date_of_birth DATE,
    medical_conditions TEXT,
    profile_picture BLOB
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {users_table_name} {users_table_definition}")

0

In [7]:
memories_table_name = "Memories"
memories_table_definition = """(
    memory_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    image BLOB,
    description TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {memories_table_name} {memories_table_definition}")

0

In [8]:
people_table_name = "People"
people_table_definition = """(
    person_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    name VARCHAR(255),
    relationship TEXT,
    description TEXT,
    picture BLOB,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {people_table_name} {people_table_definition}")

0